In [4]:
consumer_key = "" 
consumer_secret = ""

access_token = ""
access_token_secret = "" 

In [152]:
import warnings
warnings.filterwarnings("ignore")

import json
import csv
import tweepy
from textblob import TextBlob

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import pickle
from bs4 import BeautifulSoup
%matplotlib inline

import spacy
from spacy.lang.en.stop_words import STOP_WORDS as sw

In [116]:
# !pip install tweepy
model = pickle.load(open("model.pkl","rb"))
tfidf = pickle.load(open("tfidf.pkl","rb"))

In [138]:
def predict_model(msg):
    msg = [msg]
    sent = model.predict(tfidf.transform(msg))
    return sent
predict_model("Crime")

array([0], dtype=int64)

In [5]:
import tweepy

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

public_tweets = api.home_timeline() #get the tweets from homepage.


In [8]:
type(public_tweets)

tweepy.models.ResultSet

In [9]:
public_tweets

[Status(_api=<tweepy.api.API object at 0x00000217D99CA388>, _json={'created_at': 'Mon May 31 20:20:00 +0000 2021', 'id': 1399460675039154177, 'id_str': '1399460675039154177', 'text': 'Kerala assembly passes resolution seeking recall of Lakshadweep administrator\n\nhttps://t.co/ESCCvKZ73p https://t.co/2uQNkkTGIU', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/ESCCvKZ73p', 'expanded_url': 'https://www.hindustantimes.com/india-news/kerala-assembly-passes-resolution-seeking-recall-of-lakshadweep-administrator-101622437247627.html', 'display_url': 'hindustantimes.com/india-news/ker…', 'indices': [79, 102]}], 'media': [{'id': 1399229775093276678, 'id_str': '1399229775093276678', 'indices': [103, 126], 'media_url': 'http://pbs.twimg.com/media/E2sQESeVoAYOvAr.jpg', 'media_url_https': 'https://pbs.twimg.com/media/E2sQESeVoAYOvAr.jpg', 'url': 'https://t.co/2uQNkkTGIU', 'display_url': 'pic.twitter.com/2uQNkkTGIU', 'expanded_url

In [10]:
public_tweets[0]

Status(_api=<tweepy.api.API object at 0x00000217D99CA388>, _json={'created_at': 'Mon May 31 20:20:00 +0000 2021', 'id': 1399460675039154177, 'id_str': '1399460675039154177', 'text': 'Kerala assembly passes resolution seeking recall of Lakshadweep administrator\n\nhttps://t.co/ESCCvKZ73p https://t.co/2uQNkkTGIU', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/ESCCvKZ73p', 'expanded_url': 'https://www.hindustantimes.com/india-news/kerala-assembly-passes-resolution-seeking-recall-of-lakshadweep-administrator-101622437247627.html', 'display_url': 'hindustantimes.com/india-news/ker…', 'indices': [79, 102]}], 'media': [{'id': 1399229775093276678, 'id_str': '1399229775093276678', 'indices': [103, 126], 'media_url': 'http://pbs.twimg.com/media/E2sQESeVoAYOvAr.jpg', 'media_url_https': 'https://pbs.twimg.com/media/E2sQESeVoAYOvAr.jpg', 'url': 'https://t.co/2uQNkkTGIU', 'display_url': 'pic.twitter.com/2uQNkkTGIU', 'expanded_url'

In [11]:
public_tweets[0].text

'Kerala assembly passes resolution seeking recall of Lakshadweep administrator\n\nhttps://t.co/ESCCvKZ73p https://t.co/2uQNkkTGIU'

In [12]:
for tweet in public_tweets:
    print(tweet.text)

Kerala assembly passes resolution seeking recall of Lakshadweep administrator

https://t.co/ESCCvKZ73p https://t.co/2uQNkkTGIU
The decision by the Cabinet's appointments committee chaired by Prime Minister Narendra Modi was conveyed to the st… https://t.co/7VWRxPMb6E
Noted lawyer Mahesh Jethmalani on Sunday said he has been nominated to Rajya Sabha
https://t.co/W9wgsQHxWX
2 am. All alone. Streets empty. No destination.
Up for a ride? https://t.co/6DowizuvQf
#THEiX https://t.co/hMlAluYQtb
Real estate sector will contribute 10% of the GDP by 2030, according to Union Housing and Urban Affairs Secretary,… https://t.co/I4p73YqiVT
The editorial said it becomes necessary to knock on the doors of Delhi on the issue of the Maratha reservation.

https://t.co/km74gO6jhY
Shockingly, more US veterans die by suicide than in combat. This Memorial Day, let's honor those we've lost — and s… https://t.co/yVlsYtSZ0k
'India's image tarnished': Congress minister on bodies of #Covid19 patients found in rive

In [106]:
#Contarction to expansion

contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how does",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
" u ": " you ",
" ur ": " your ",
" n ": " and ",
"won't": "would not",
'dis': 'this',
'bak': 'back',
'brng': 'bring'}

def cont_to_exp(x):
    if type(x) is str:
        for key in contractions:
            value = contractions[key]
            x = x.replace(key, value)
        return x
    else:
        return x

### Tracking Keywords

In [163]:
#override tweepy.StreamListener to add logic to on_status

usa = 0
china = 0
keyword = ["usa","china"]

with open("sentiment.csv","w") as f:
    writer = csv.DictWriter(f,fieldnames= keyword)
    writer.writeheader()

class MyStreamListener(tweepy.StreamListener):
    
    def on_data(self,data):
        tweets = json.loads(data)
        try:
            twt = str(tweets["text"]).lower()
#             print("1")
            twt = cont_to_exp(twt)
#             print("2")
            twt = re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , twt)
#             print("3")
            twt = re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+)',"",twt)
#             print("4")
            twt = re.sub('\brt\b','', twt).strip()
#             print("5")
            twt = BeautifulSoup(twt,"lxml").get_text().strip()
#             print("6")
            twt = re.sub('[^\w ]+',"", twt).strip()
#             print("7")
            twt = " ".join(twt.split())
#             print("8")
#             --------------- TEXTBLOB SENTIMENT CLASSIFICATION -------------------
#             blob = TextBlob(twt)
# #             print(blob)
#             sent = bolb.sentiment.polarity
#             print(sent)
#             sent = predict_model(twt) #will give u a list either [0] or [1]
#             print(sent)
#            ------------------------------------------------------------------------
#            ----------------- SVM MODEL AND TFIDF ----------------------------------
            
            global china, usa
            
            if "usa" in twt and "china" not in twt:
                sent = predict_model(twt)[0]
                usa += sent
            elif "china" in twt and "usa" not in twt:
                sent = predict_model(twt)[0]
                china += sent
            else:
                pass
            
#             print("USA : ",usa) #will show the count of positive sents
#             print("CHINA : ",china)

            with open("sentiment.csv","a") as f:
                writer = csv.DictWriter(f,fieldnames= keyword)
                info = {"usa": usa, "china": china}
                writer.writerow(info)
        except:
            pass
    
    def on_error(self,status_code):
        if status_code == 420:
            print("ERROR 420")
            return False

    def on_status(self, status):
        print(status.text)

In [164]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)

In [166]:
myStream.filter(track = keyword)